## CELL 1: Setup and Imports

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict
import sys
import os


from XAI_engine.xai_engine import FakeReviewXAI

print("Imports successful!")

Imports successful!


## CELL 2: Load Dataset

In [8]:
df = pd.read_csv('../data/fake_reviews_dataset.csv')

print(f"Total reviews: {len(df):,}")
print(f"\nLabel distribution:")
print(df['label'].value_counts())
print(f"\nCG = Computer Generated (FAKE)")
print(f"OR = Original (GENUINE)")

Total reviews: 40,432

Label distribution:
label
CG    20216
OR    20216
Name: count, dtype: int64

CG = Computer Generated (FAKE)
OR = Original (GENUINE)


## CELL 3: Prepare Validation Data

In [5]:
n_samples = 50

fake_samples = df[df['label'] == 'CG'].sample(n=n_samples, random_state=42)
genuine_samples = df[df['label'] == 'OR'].sample(n=n_samples, random_state=42)

validation_df = pd.concat([fake_samples, genuine_samples])
validation_df['is_fake'] = validation_df['label'] == 'CG'
validation_df = validation_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Validation set: {len(validation_df)} reviews")
print(f"Fake: {validation_df['is_fake'].sum()}")
print(f"Genuine: {(~validation_df['is_fake']).sum()}")

Validation set: 100 reviews
Fake: 50
Genuine: 50


## CELL 4: Tuning Function
Purpose: Test XAI engine on reviews and record predictions

In [9]:
def tune_thresholds(reviews_df, xai_engine):
    
    """Test XAI engine on labeled data"""
    
    results = []
    
    for idx, row in reviews_df.iterrows():
        try:
            analysis = xai_engine.analyze_review(row['text_'])
            
            predicted_fake = analysis['verdict'] in ['LIKELY FAKE', 'SUSPICIOUS']
            actual_fake = row['is_fake']
            
            results.append({
                'actual_fake': actual_fake,
                'predicted_fake': predicted_fake,
                'verdict': analysis['verdict'],
                'confidence': analysis['confidence'],
                'flag_count': analysis['flag_count'],
                'correct': predicted_fake == actual_fake
            })
        except Exception as e:
            print(f"Error on review {idx}: {e}")
            continue
    
    return results

print("Tuning function ready")

Tuning function ready


## CELL 5: Calculate Metrics function
Purpose: Compute accuracy, precision, recall, F1 score

In [10]:
def calculate_metrics(results):
    
    """Calculate performance metrics"""
    
    tp = sum(1 for r in results if r['actual_fake'] and r['predicted_fake'])
    fp = sum(1 for r in results if not r['actual_fake'] and r['predicted_fake'])
    tn = sum(1 for r in results if not r['actual_fake'] and not r['predicted_fake'])
    fn = sum(1 for r in results if r['actual_fake'] and not r['predicted_fake'])
    
    total = len(results)
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'false_negatives': fn,
        'total_samples': total
    }

print("Metrics function ready")

Metrics function ready


## cell 6 : Optimization

### CELL 6A: Stage 1 - Optimize Tier 1 Features

In [15]:
# ========================================
# CELL 6A: Stage 1 - Optimize Tier 1 Features
# ========================================
from itertools import product

print("Two-Stage Grid Search for Optimal Thresholds")
print("="*60)
print("\nSTAGE 1: Optimizing Tier 1 Features")
print("Testing: Sentiment, Word Count, Adj/Noun Ratio, First-Person Ratio\n")

# Define Tier 1 ranges
sentiment_range = [0.75, 0.80, 0.85, 0.90, 0.95]
word_min_range = [10, 15, 20]
adj_noun_range = [2.0, 2.5, 3.0]
first_person_range = [0.10, 0.15, 0.20]

total_stage1 = len(sentiment_range) * len(word_min_range) * len(adj_noun_range) * len(first_person_range)
print(f"Stage 1 combinations: {total_stage1}")

stage1_results = []
counter = 0

for sent, word_min, adj_noun, first_p in product(
    sentiment_range, word_min_range, adj_noun_range, first_person_range
):
    counter += 1
    
    if counter % 20 == 0: #Print progress every 20 iterations
        print(f"  Progress: {counter}/{total_stage1} ({counter/total_stage1:.0%})")
    
    # Create config with FIXED Tier 2 values
    config = {
        'name': f'Stage1_{counter}',
        'SENTIMENT_EXTREME': sent,
        'WORD_COUNT_MIN': word_min,
        'WORD_COUNT_MAX': 200,
        'ADJ_NOUN_RATIO': adj_noun,
        'FIRST_PERSON_RATIO': first_p,
        'SPAM_KEYWORD_MIN': 3,
        'CAPS_RATIO_MAX': 0.20,
        'EXCESSIVE_PUNCT_MIN': 3,
        'UNIQUENESS_RATIO_MIN': 0.60
    }
    
    # Test configuration
    xai = FakeReviewXAI()
    config_thresholds = {k: v for k, v in config.items() if k != 'name'} #creates threshold dictionary without 'name' to update object thresholds
    xai.thresholds.update(config_thresholds)
    
    results = tune_thresholds(validation_df, xai)
    metrics = calculate_metrics(results)
    
    config_result = config.copy()
    config_result.update(metrics)
    stage1_results.append(config_result)

# Get best Tier 1 configuration
stage1_df = pd.DataFrame(stage1_results)
stage1_df = stage1_df.sort_values('accuracy', ascending=False, ignore_index=True)
best_tier1 = stage1_df.iloc[0]

print(f"\nStage 1 Complete!")
print(f"Best Tier 1 Accuracy: {best_tier1['accuracy']:.2%}")
print(f"\nOptimal Tier 1 Thresholds:")
print(f"  SENTIMENT_EXTREME:    {best_tier1['SENTIMENT_EXTREME']}")
print(f"  WORD_COUNT_MIN:       {best_tier1['WORD_COUNT_MIN']}")
print(f"  ADJ_NOUN_RATIO:       {best_tier1['ADJ_NOUN_RATIO']}")
print(f"  FIRST_PERSON_RATIO:   {best_tier1['FIRST_PERSON_RATIO']}")

Two-Stage Grid Search for Optimal Thresholds

STAGE 1: Optimizing Tier 1 Features
Testing: Sentiment, Word Count, Adj/Noun Ratio, First-Person Ratio

Stage 1 combinations: 135
  Progress: 20/135 (15%)
  Progress: 40/135 (30%)
  Progress: 60/135 (44%)
  Progress: 80/135 (59%)
  Progress: 100/135 (74%)
  Progress: 120/135 (89%)

Stage 1 Complete!
Best Tier 1 Accuracy: 60.00%

Optimal Tier 1 Thresholds:
  SENTIMENT_EXTREME:    0.95
  WORD_COUNT_MIN:       20
  ADJ_NOUN_RATIO:       2.0
  FIRST_PERSON_RATIO:   0.1


### CELL 6B: Stage 2 - Optimize Tier 2 Features

In [16]:
# ========================================
# CELL 6B: Stage 2 - Optimize Tier 2 Features
# ========================================
print("\n" + "="*60)
print("STAGE 2: Optimizing Tier 2 Features")
print("Testing: Spam Keywords, Caps, Punctuation, Redundancy\n")

# Define Tier 2 ranges
spam_keyword_range = [2, 3, 4]
caps_ratio_range = [0.15, 0.20, 0.25]
punct_range = [2, 3, 4]
uniqueness_range = [0.55, 0.60, 0.65]

total_stage2 = len(spam_keyword_range) * len(caps_ratio_range) * len(punct_range) * len(uniqueness_range)
print(f"Stage 2 combinations: {total_stage2}")

stage2_results = []
counter = 0

for spam, caps, punct, unique in product(
    spam_keyword_range, caps_ratio_range, punct_range, uniqueness_range
):
    counter += 1
    
    if counter % 15 == 0:
        print(f"  Progress: {counter}/{total_stage2} ({counter/total_stage2:.0%})")
    
    # Create config with OPTIMAL Tier 1 + varying Tier 2
    config = {
        'name': f'Stage2_{counter}',
        # Tier 1 - FIXED at optimal from Stage 1
        'SENTIMENT_EXTREME': best_tier1['SENTIMENT_EXTREME'],
        'WORD_COUNT_MIN': best_tier1['WORD_COUNT_MIN'],
        'WORD_COUNT_MAX': 200,
        'ADJ_NOUN_RATIO': best_tier1['ADJ_NOUN_RATIO'],
        'FIRST_PERSON_RATIO': best_tier1['FIRST_PERSON_RATIO'],
        # Tier 2 - SEARCHING
        'SPAM_KEYWORD_MIN': spam,
        'CAPS_RATIO_MAX': caps,
        'EXCESSIVE_PUNCT_MIN': punct,
        'UNIQUENESS_RATIO_MIN': unique
    }
    
    # Test configuration
    xai = FakeReviewXAI()
    config_thresholds = {k: v for k, v in config.items() if k != 'name'}
    xai.thresholds.update(config_thresholds)
    
    results = tune_thresholds(validation_df, xai)
    metrics = calculate_metrics(results)
    
    config_result = config.copy()
    config_result.update(metrics)
    stage2_results.append(config_result)

# Get best overall configuration
stage2_df = pd.DataFrame(stage2_results)
stage2_df = stage2_df.sort_values('accuracy', ascending=False, ignore_index=True)
best_overall = stage2_df.iloc[0]

print(f"\nStage 2 Complete!")
print(f"Best Overall Accuracy: {best_overall['accuracy']:.2%}")
print(f"\nOptimal Tier 2 Thresholds:")
print(f"  SPAM_KEYWORD_MIN:     {best_overall['SPAM_KEYWORD_MIN']}")
print(f"  CAPS_RATIO_MAX:       {best_overall['CAPS_RATIO_MAX']}")
print(f"  EXCESSIVE_PUNCT_MIN:  {best_overall['EXCESSIVE_PUNCT_MIN']}")
print(f"  UNIQUENESS_RATIO_MIN: {best_overall['UNIQUENESS_RATIO_MIN']}")


STAGE 2: Optimizing Tier 2 Features
Testing: Spam Keywords, Caps, Punctuation, Redundancy

Stage 2 combinations: 81
  Progress: 15/81 (19%)
  Progress: 30/81 (37%)
  Progress: 45/81 (56%)
  Progress: 60/81 (74%)
  Progress: 75/81 (93%)

Stage 2 Complete!
Best Overall Accuracy: 60.00%

Optimal Tier 2 Thresholds:
  SPAM_KEYWORD_MIN:     4
  CAPS_RATIO_MAX:       0.25
  EXCESSIVE_PUNCT_MIN:  3
  UNIQUENESS_RATIO_MIN: 0.6


### CELL 6C: Combine Results

In [17]:
# ========================================
# CELL 6C: Combine and Save All Results
# ========================================
# Combine all results
all_results = pd.concat([stage1_df, stage2_df], ignore_index=True)
all_results = all_results.sort_values('accuracy', ascending=False)
all_results.to_csv('../data/grid_search_results.csv', index=False)

# Save as grid_df for later cells
grid_df = all_results

print("="*60)
print("TWO-STAGE GRID SEARCH COMPLETE!")
print("="*60)
print(f"Total configurations tested: {total_stage1 + total_stage2}")
print(f"Best accuracy achieved: {best_overall['accuracy']:.2%}\n")
print("Top 10 configurations:")
print(grid_df[['SENTIMENT_EXTREME', 'WORD_COUNT_MIN', 'ADJ_NOUN_RATIO', 
               'FIRST_PERSON_RATIO', 'SPAM_KEYWORD_MIN', 'CAPS_RATIO_MAX',
               'accuracy', 'f1_score']].head(10))

print("\nFull results saved to: data/grid_search_results.csv")

TWO-STAGE GRID SEARCH COMPLETE!
Total configurations tested: 216
Best accuracy achieved: 60.00%

Top 10 configurations:
     SENTIMENT_EXTREME  WORD_COUNT_MIN  ADJ_NOUN_RATIO  FIRST_PERSON_RATIO  \
0                 0.95              20             2.0                 0.1   
142               0.95              20             2.0                 0.1   
141               0.95              20             2.0                 0.1   
140               0.95              20             2.0                 0.1   
139               0.95              20             2.0                 0.1   
138               0.95              20             2.0                 0.1   
152               0.95              20             2.0                 0.1   
135               0.95              20             2.0                 0.1   
151               0.95              20             2.0                 0.1   
150               0.95              20             2.0                 0.1   

     SPAM_KEYWORD_MIN

## CELL 7: Save Top 5 to threshold_tuning.csv

In [19]:
top_5 = grid_df.head(5)

column_order = [
    'name', 'SENTIMENT_EXTREME', 'WORD_COUNT_MIN', 'WORD_COUNT_MAX',
    'ADJ_NOUN_RATIO', 'FIRST_PERSON_RATIO', 'SPAM_KEYWORD_MIN',
    'CAPS_RATIO_MAX', 'EXCESSIVE_PUNCT_MIN', 'UNIQUENESS_RATIO_MIN',
    'accuracy', 'precision', 'recall', 'f1_score',
    'true_positives', 'false_positives', 'true_negatives', 'false_negatives',
    'total_samples'
]

top_5 = top_5[column_order]
top_5.to_csv('../data/threshold_tuning.csv', index=False)

print("Top 5 saved to: data/threshold_tuning.csv")
print("\nTop 5 Results:")
print(top_5[['name', 'accuracy', 'precision', 'recall', 'f1_score']])

Top 5 saved to: data/threshold_tuning.csv

Top 5 Results:
           name  accuracy  precision  recall  f1_score
0    Stage1_127       0.6   0.678571    0.38  0.487179
142   Stage2_71       0.6   0.678571    0.38  0.487179
141   Stage2_65       0.6   0.678571    0.38  0.487179
140   Stage2_62       0.6   0.678571    0.38  0.487179
139   Stage2_59       0.6   0.678571    0.38  0.487179


## CELL 8: Display Best Configuration

In [20]:
best_idx = grid_df['accuracy'].idxmax()
best_config = grid_df.iloc[best_idx]

print("="*60)
print("OPTIMAL CONFIGURATION (Found by Grid Search)")
print("="*60)
print(f"\nPerformance:")
print(f"  Accuracy:  {best_config['accuracy']:.2%}")
print(f"  Precision: {best_config['precision']:.2%}")
print(f"  Recall:    {best_config['recall']:.2%}")
print(f"  F1 Score:  {best_config['f1_score']:.2%}")
print(f"\nThresholds:")
print(f"  SENTIMENT_EXTREME:      {best_config['SENTIMENT_EXTREME']}")
print(f"  WORD_COUNT_MIN:         {best_config['WORD_COUNT_MIN']}")
print(f"  WORD_COUNT_MAX:         {best_config['WORD_COUNT_MAX']}")
print(f"  ADJ_NOUN_RATIO:         {best_config['ADJ_NOUN_RATIO']}")
print(f"  FIRST_PERSON_RATIO:     {best_config['FIRST_PERSON_RATIO']}")
print(f"  SPAM_KEYWORD_MIN:       {best_config['SPAM_KEYWORD_MIN']}")
print(f"  CAPS_RATIO_MAX:         {best_config['CAPS_RATIO_MAX']}")
print(f"  EXCESSIVE_PUNCT_MIN:    {best_config['EXCESSIVE_PUNCT_MIN']}")
print(f"  UNIQUENESS_RATIO_MIN:   {best_config['UNIQUENESS_RATIO_MIN']}")

OPTIMAL CONFIGURATION (Found by Grid Search)

Performance:
  Accuracy:  60.00%
  Precision: 67.86%
  Recall:    38.00%
  F1 Score:  48.72%

Thresholds:
  SENTIMENT_EXTREME:      0.95
  WORD_COUNT_MIN:         20
  WORD_COUNT_MAX:         200
  ADJ_NOUN_RATIO:         2.0
  FIRST_PERSON_RATIO:     0.1
  SPAM_KEYWORD_MIN:       3
  CAPS_RATIO_MAX:         0.2
  EXCESSIVE_PUNCT_MIN:    3
  UNIQUENESS_RATIO_MIN:   0.6


## CELL 11: Generate config.py Code
updates config.py with optimal thresholds

In [24]:
# ========================================
# CELL 11: Auto-Update config.py
# ========================================
import os


# Generate the new config content
new_config_content = f'''"""
XAI Engine Configuration
Centralized thresholds for all features (Tier 1 + Tier 2)
Auto-generated by threshold_tuning.ipynb
"""

# ============================================================================
# ALL THRESHOLDS (TIER 1 + TIER 2) - OPTIMIZED
# ============================================================================

THRESHOLDS = {{
    # TIER 1: Essential Features
    'SENTIMENT_EXTREME': {best_config['SENTIMENT_EXTREME']},
    'WORD_COUNT_MIN': {int(best_config['WORD_COUNT_MIN'])},
    'WORD_COUNT_MAX': {int(best_config['WORD_COUNT_MAX'])},
    'ADJ_NOUN_RATIO': {best_config['ADJ_NOUN_RATIO']},
    'FIRST_PERSON_RATIO': {best_config['FIRST_PERSON_RATIO']},
    
    # TIER 2: Important Features
    'SPAM_KEYWORD_MIN': {int(best_config['SPAM_KEYWORD_MIN'])},
    'CAPS_RATIO_MAX': {best_config['CAPS_RATIO_MAX']},
    'EXCESSIVE_PUNCT_MIN': {int(best_config['EXCESSIVE_PUNCT_MIN'])},
    'UNIQUENESS_RATIO_MIN': {best_config['UNIQUENESS_RATIO_MIN']}
}}


def load_config():
    """Return a copy of thresholds"""
    return THRESHOLDS.copy()


def update_thresholds(new_thresholds):
    """Update thresholds (used by threshold tuning)"""
    THRESHOLDS.update(new_thresholds)
'''

# Write to config.py
config_path = '../xai_engine/config.py'

try:
    with open(config_path, 'w') as f:
        f.write(new_config_content)
    
    print("✅ config.py successfully updated!")
    print(f"\nFile location: {os.path.abspath(config_path)}")
    
except Exception as e:
    print(f"❌ Error updating config.py: {e}")
    print("\nManual update required. Copy this code:\n")
    print(new_config_content)

# Display the optimal configuration
print("\n" + "="*60)
print("🏆 OPTIMAL CONFIGURATION")
print("="*60)
print(f"\nPerformance:")
print(f"  Accuracy:  {best_config['accuracy']:.2%}")
print(f"  Precision: {best_config['precision']:.2%}")
print(f"  Recall:    {best_config['recall']:.2%}")
print(f"  F1 Score:  {best_config['f1_score']:.2%}")

print(f"\nOptimal Thresholds (now in config.py):")
print(f"  SENTIMENT_EXTREME:      {best_config['SENTIMENT_EXTREME']}")
print(f"  WORD_COUNT_MIN:         {int(best_config['WORD_COUNT_MIN'])}")
print(f"  WORD_COUNT_MAX:         {int(best_config['WORD_COUNT_MAX'])}")
print(f"  ADJ_NOUN_RATIO:         {best_config['ADJ_NOUN_RATIO']}")
print(f"  FIRST_PERSON_RATIO:     {best_config['FIRST_PERSON_RATIO']}")
print(f"  SPAM_KEYWORD_MIN:       {int(best_config['SPAM_KEYWORD_MIN'])}")
print(f"  CAPS_RATIO_MAX:         {best_config['CAPS_RATIO_MAX']}")
print(f"  EXCESSIVE_PUNCT_MIN:    {int(best_config['EXCESSIVE_PUNCT_MIN'])}")
print(f"  UNIQUENESS_RATIO_MIN:   {best_config['UNIQUENESS_RATIO_MIN']}")

✅ config.py successfully updated!

File location: c:\Users\Amr\Documents\GitHub\Team-C-repository\xai_engine\config.py

🏆 OPTIMAL CONFIGURATION

Performance:
  Accuracy:  60.00%
  Precision: 67.86%
  Recall:    38.00%
  F1 Score:  48.72%

Optimal Thresholds (now in config.py):
  SENTIMENT_EXTREME:      0.95
  WORD_COUNT_MIN:         20
  WORD_COUNT_MAX:         200
  ADJ_NOUN_RATIO:         2.0
  FIRST_PERSON_RATIO:     0.1
  SPAM_KEYWORD_MIN:       3
  CAPS_RATIO_MAX:         0.2
  EXCESSIVE_PUNCT_MIN:    3
  UNIQUENESS_RATIO_MIN:   0.6
